<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/00_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [46]:
#| default_exp colab

# Google Colab utils
> Deal with colab specific format

In [47]:
#| export
from pathlib import Path
from IPython import get_ipython
from IPython.display import display, Markdown

from fastcore.basics import ifnone

from execnb.nbio import NbCell, read_nb

from nb_helpers.utils import git_main_name, git_origin_repo, git_local_repo, search_cell, git_current_branch

In [48]:
this_nb = Path("00_colab.ipynb")
notebook = read_nb(this_nb)

In [49]:
this_nb.resolve()

Path('/Users/tcapelle/work/nb_helpers/nbs/00_colab.ipynb')

In [50]:
git_origin_repo(this_nb.resolve())

'wandb/nb_helpers'

In [51]:
#| export
def get_colab_url(fname, branch):
    "Get git repo url, to append to colab"
    fname = Path(fname).resolve()
    github_repo = git_origin_repo(fname)
    fname = fname.relative_to(git_local_repo(fname))

    return f"https://colab.research.google.com/github/{github_repo}/blob/{branch}/{str(fname)}"

In [52]:
get_colab_url(this_nb, branch="master")

'https://colab.research.google.com/github/wandb/nb_helpers/blob/master/nbs/00_colab.ipynb'

In [53]:
#| exporti
_badge_meta = {"id": "view-in-github", "colab_type": "text"}

In [54]:
#| export
def _create_colab_cell(url, meta={}, tracker=None):
    "Creates a notebook cell with the `Open In Colab` badge"
    tracker = ifnone(tracker, "")
    data = {
        "cell_type": "markdown",
        "metadata": meta,
        "source": f'<a href="{url}" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>\n'
        + tracker,
    }
    return NbCell(-1, data)

In [55]:
url = get_colab_url(this_nb, branch="master")
cell = _create_colab_cell(url)
display(Markdown(cell["source"]))

<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/master/nbs/00_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [56]:
#| export
def has_colab_badge(nb):
    "Check if notebook has colab badge, returns the cell position, -1 if not present"
    for i, cell in enumerate(nb["cells"]):
        if search_cell(cell, "Open In Colab") and cell['cell_type'] == 'markdown':
            return i
    return -1

In [57]:
assert has_colab_badge(notebook) == 0

In [58]:
def remove_colab_badges(notebook):
    "Remove all Colab badges from the notebook"
    for cell in notebook['cells']:
        if cell['cell_type'] == 'markdown' and search_cell(cell, "colab.research.google"):
            print(f"Removing badge from cell {cell['source']}")
            lines = cell['source'].split('\n')
            new_lines = [line for line in lines if "colab.research.google" not in line]
            cell['source'] = '\n'.join(new_lines)
    return notebook

In [59]:
non_badged_nb = remove_colab_badges(notebook)
assert has_colab_badge(non_badged_nb) == -1, "Badge are still there?!"

Removing badge from cell <a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/00_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In [60]:
#| export
def create_colab_badge_cell(fname, branch=None, meta={}, tracker=None):
    "Create a colab badge cell from `fname`"
    # get main/master name
    branch = ifnone(branch, git_main_name(fname))
    url = get_colab_url(fname, branch)
    colab_cell = _create_colab_cell(url, meta, tracker)
    return colab_cell

In [61]:
create_colab_badge_cell(this_nb)

```json
{ 'cell_type': 'markdown',
  'idx_': -1,
  'metadata': {},
  'source': '<a '
            'href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/00_colab.ipynb" '
            'target="_parent"><img '
            'src="https://colab.research.google.com/assets/colab-badge.svg" '
            'alt="Open In Colab"/></a>\n'}
```

In [62]:
#| export
def add_colab_badge(notebook, fname, branch=None, idx=0, meta=_badge_meta, tracker=None):
    "Add a badge to Open In Colab in the `idx` cell"
    notebook = remove_colab_badges(notebook)
    colab_cell = create_colab_badge_cell(fname, branch, meta, tracker)
    notebook["cells"].insert(idx, colab_cell)
    return notebook

In [63]:
nb_with_colab_badge = add_colab_badge(notebook, fname=this_nb)

it should have the colab badge in the first cell now

In [64]:
assert has_colab_badge(nb_with_colab_badge) == 0

Let's also activate the GPU on colab

In [65]:
d = dict(a=1, b=2)

In [66]:
d.update({"c": 3})

In [67]:
#| exporti
_colab_meta = {
    "colab": {
        "include_colab_link": True, 
        "toc_visible": True,
        "provenance": []
        },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
      },
    "language_info": {
      "name": "python"
      },
    "accelerator": "GPU",
    "gpuClass": "standard"
    }


In [68]:
#| export
def add_colab_metadata(notebook, meta=_colab_meta):
    "Adds GPU and colab meta to `notebook`"
    if "accelerator" in notebook: # old colaboratory standard
        del notebook["accelerator"]
    notebook["metadata"].update(_colab_meta)
    return notebook

In [69]:
add_colab_metadata(notebook)["metadata"]

```json
{ 'accelerator': 'GPU',
  'colab': {'include_colab_link': True, 'provenance': [], 'toc_visible': True},
  'gpuClass': 'standard',
  'kernelspec': {'display_name': 'Python 3', 'name': 'python3'},
  'language_info': {'name': 'python'}}
```

## Export -

In [70]:
#|hide
from nbdev import *
nbdev_export()